In [8]:
#importing useful packages
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.io import ascii
import astropy.time
from astropy.time import Time
import astropy.table
from astropy.table import Table, Column, MaskedColumn
from astroplan import Observer
from astroquery.jplhorizons import Horizons
import pandas as pd
import csv
import astropy.time
import dateutil.parser

### Reading CSV file of dates targets were observed
raw_target_info = pd.read_csv("look_table.csv")

### Creating new dataframe with first date each target was observed
subset_df = raw_target_info[["Target","Date"]]
grouped_and_first_df = subset_df.groupby(["Target"]).first()
new_df = grouped_and_first_df.reset_index()
new_df.columns = ["Name","N"]

### 
target_location = '474' # Mt. John Observatory MPC code

obsDate_lst = []
for i in new_df.N:
    obsDate_lst.append(i)

targetname_lst = []
for i in new_df.Name:
    targetname_lst.append(i)

# need to convert epoch date to julian date
jd_obsDate_lst = []
for i in obsDate_lst:
    dt = dateutil.parser.parse(obsDate_lst[0]) 
    time = astropy.time.Time(dt)
    jd_obsDate_lst.append(time.jd)


# statue_of_liberty = {'lon': -74.0466891,
#                      'lat': 40.6892534,
#                      'elevation': 0.093}

# obj = Horizons(id='Eris',epochs=jd_obsDate_lst[0])
# print(obj.elements())
# print(obj.location)



# #need to query information from JPL Horizons database
eph_lst = []
el_lst = []
obj_lst = []
for index,jdDate in enumerate(jd_obsDate_lst):
    if targetname_lst[index] == "C/2018 F4":
        obj = Horizons(id='90004395', location=target_location,epochs=jdDate)
    else:
        obj = Horizons(id=targetname_lst[index], epochs=jdDate)
        obj_lst.append(obj)
        eph_lst.append(obj.ephemerides()) #from eph, only need 'r' heliocentric distance
        el_lst.append(obj.elements()) #from elems, need a0 (semimajor axis), q, and T (perihelion date)

In [9]:
print(el_lst[0][0])

    targetname    datetime_jd          datetime_str            H    G           e                 q                 incl             Omega              w               Tp_jd               n                 M                  nu                  a                      Q                     P          
       ---             d                   ---                mag  ---         ---                AU                deg               deg              deg                d              deg / d             deg                deg                  AU                     AU                    d          
----------------- ----------- ------------------------------ ----- ---- ----------------- ------------------ ----------------- ----------------- ---------------- ----------------- ----------------- ------------------ ----------------- ---------------------- --------------------- ---------------------
189040 (2000 MU1)   2459235.5 A.D. 2021-Jan-21 00:00:00.0000 19.88 0.15 7205.508971615393 0.09

In [22]:
final_array = []
for index,target in enumerate(eph_lst):
    final_array.append([el_lst[index]['a'][0],eph_lst[index]['r'][0],
                        el_lst[index]['q'][0],el_lst[index]['Tp_jd'][0]])

In [27]:
eph_el_df = pd.DataFrame(final_array,columns=['a','r','q','T'])
eph_el_df.head(4)

,a,r,q,T
0,-0.000013,1.023791,0.091482,2.459241e+06
1,-0.000016,1.104592,0.102607,2.459245e+06
2,-0.000490,0.983340,0.102455,2.459180e+06
3,-0.000141,1.022337,0.011291,2.459254e+06


In [31]:
final_new_df = pd.concat([new_df,eph_el_df],axis=1)
final_new_df.head(5)

,Name,N,a,r,q,T
0,189040,21/01/2021,-0.000013,1.023791,0.091482,2.459241e+06
1,2009 CD,11/02/2021,-0.000016,1.104592,0.102607,2.459245e+06
2,2020 UB5,11/02/2021,-0.000490,0.983340,0.102455,2.459180e+06
3,2021 CA1,11/02/2021,-0.000141,1.022337,0.011291,2.459254e+06
4,2021 CD1,12/02/2021,-0.000010,1.139992,0.051302,2.459260e+06


In [33]:
final_new_df = final_new_df[['Name','a','N','r','q','T']]
final_new_df.head(5)

,Name,a,N,r,q,T
0,189040,-0.000013,21/01/2021,1.023791,0.091482,2.459241e+06
1,2009 CD,-0.000016,11/02/2021,1.104592,0.102607,2.459245e+06
2,2020 UB5,-0.000490,11/02/2021,0.983340,0.102455,2.459180e+06
3,2021 CA1,-0.000141,11/02/2021,1.022337,0.011291,2.459254e+06
4,2021 CD1,-0.000010,12/02/2021,1.139992,0.051302,2.459260e+06


In [10]:
#now we need to make the JPL data and the FITS data into tables
final_r_lst = []
for target in eph_lst:
    final_r_lst.append(target['r'][0])

In [11]:
print(final_r_lst)

[1.023791360949, 1.104592496759, 0.983340023274, 1.022337473413, 1.139991851076, 1.083627970415, 1.019256288507, 0.992017928236, 1.031319318383, 1.04734870929, 6.237595579648, 1.413173122935, 7.95430957502, 2.85525765412, 2.229828039621, 6.230638919391, 5.226928081046]


In [12]:
final_a0_lst = []
final_q_lst = []
final_T_lst = []
for target in el_lst:
    final_a0_lst.append(target['a'][0]) 
    final_q_lst.append(target['q'][0])
    final_T_lst.append(target['Tp_jd'][0])

In [13]:
print(final_a0_lst)
print(final_q_lst)
print(final_T_lst)

[-1.269786257689621e-05, -1.582153576294483e-05, -0.0004903242242290665, -0.000141160640325975, -9.795714123171923e-06, -3.508603018446807e-05, -5.807100685275533e-05, -3.281811186662119e-05, -1.533055015280568e-05, -3.193896527215872e-05, -4.885987169310872e-06, -9.933732605506793e-07, -3.115936583840077e-06, -3.258038291054867e-06, -8.568433316583811e-07, -6.356857989676344e-06, -7.393996329128375e-06]
[0.0914818648555881, 0.1026071117739761, 0.102455354005726, 0.01129115889836261, 0.05130165864243345, 0.01556908603898099, 0.02934861540793406, 0.02352240347185621, 0.0780814201946056, 0.2210113043571349, 6.220852488759641, 0.4960130519828689, 4.928463953963444, 1.96579552427586, 1.878752050572924, 2.926078370623832, 1.507295891016705]
[2459240.592245275, 2459245.404798997, 2459179.537558207, 2459254.176984094, 2459260.28539629, 2459255.875013671, 2459251.841910695, 2459249.232853253, 2459247.052381009, 2459231.093043865, 2459155.87785025, 2459249.722663001, 2459550.129580839, 2459343.

In [21]:
df2 = pd.DataFrame([final_a0_lst, final_r_lst, final_q_lst, final_T_lst])
print(df2)

             0             1             2             3             4   \
0 -1.269786e-05 -1.582154e-05 -4.903242e-04 -1.411606e-04 -9.795714e-06   
1  1.023791e+00  1.104592e+00  9.833400e-01  1.022337e+00  1.139992e+00   
2  9.148186e-02  1.026071e-01  1.024554e-01  1.129116e-02  5.130166e-02   
3  2.459241e+06  2.459245e+06  2.459180e+06  2.459254e+06  2.459260e+06   

             5             6             7             8             9   \
0 -3.508603e-05 -5.807101e-05 -3.281811e-05 -1.533055e-05 -3.193897e-05   
1  1.083628e+00  1.019256e+00  9.920179e-01  1.031319e+00  1.047349e+00   
2  1.556909e-02  2.934862e-02  2.352240e-02  7.808142e-02  2.210113e-01   
3  2.459256e+06  2.459252e+06  2.459249e+06  2.459247e+06  2.459231e+06   

             10            11            12            13            14  \
0 -4.885987e-06 -9.933733e-07 -3.115937e-06 -3.258038e-06 -8.568433e-07   
1  6.237596e+00  1.413173e+00  7.954310e+00  2.855258e+00  2.229828e+00   
2  6.220852e+00  4.9601

In [15]:
yolo = final_el_lst + final_eph_lst
print(final_el_lst[0])
print(final_eph_lst[0])
print(yolo)

NameError: name 'final_el_lst' is not defined

In [ ]:
df1 = pd.DataFrame({'a':final_el_lst,
                    'b':final_eph_lst})
print(df1.head(1))

In [17]:
my_arr = np.array([[1,2,3],
                   [4,5,6],
                   [7,8,9]])

my_df = pd.DataFrame(my_arr,columns=['col1','col2','col3'])

In [18]:
print(my_df)

   col1  col2  col3
0     1     2     3
1     4     5     6
2     7     8     9
